In [ ]:
# 1. Imports
import sys
sys.path.append("../src")
from data_processing import *

import pandas as pd

# ✅ Toggle: fast mode skips plotting to save time on large datasets
FAST_MODE = True  # Set False to run full visualizations

# Define load_data function directly in notebook
def load_data(path="../data/raw/data.csv"):
    return pd.read_csv(path)

# -----------------------------------------------
# 🚀 Main Notebook Workflow

# Load Data
df = load_data()

# Overview
overview(df)

# Summary Stats
summary_stats = summary_statistics(df)
print(summary_stats)

# Visualizations
if not FAST_MODE:
    print("📊 Plotting distributions...")
    plot_numerical_distributions(df)
    plot_categorical_distributions(df)

# Correlation
print("🔗 Correlation Matrix")
correlation_matrix(df)

# Missing values
missing = missing_values(df)
print("Missing Values:\n", missing[missing > 0])

# Outliers
outliers = detect_outliers(df)
print("Outliers:\n", outliers)

# Boxplots
if not FAST_MODE:
    plot_boxplots(df)

# Save Cleaned Data
save_processed_data(df)


: 